In [1]:
!pip install -q ultralytics gdown opencv-python moviepy scipy tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.5 MB/s eta 0:00:00


In [2]:
import gdown

gdown.download(id='1-5fOSHOSB9UXyP_enOoZNAMScrePVcMD', output='best.pt', quiet=False)
gdown.download(id='1a_50owJBTszIG3fHXGhlXMlXXuCDZ9Cu', output='broadcast.mp4', quiet=False)
gdown.download(id='1qnTp2Ij1HD7h9R3z-fPUkwmrg0DN0a_c', output='tacticam.mp4', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1-5fOSHOSB9UXyP_enOoZNAMScrePVcMD
From (redirected): https://drive.google.com/uc?id=1-5fOSHOSB9UXyP_enOoZNAMScrePVcMD&confirm=t&uuid=547f53c0-3fdf-4cdd-bc47-ac2d8d6723c1
To: /content/best.pt
100%|██████████| 195M/195M [00:01<00:00, 185MB/s]
Downloading...
From: https://drive.google.com/uc?id=1a_50owJBTszIG3fHXGhlXMlXXuCDZ9Cu
To: /content/broadcast.mp4
100%|██████████| 9.23M/9.23M [00:00<00:00, 41.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qnTp2Ij1HD7h9R3z-fPUkwmrg0DN0a_c
To: /content/tacticam.mp4
100%|██████████| 10.5M/10.5M [00:00<00:00, 31.5MB/s]


'tacticam.mp4'

In [3]:
from ultralytics import YOLO
import cv2
import os
from tqdm import tqdm

def run_detection(video_path, model_path, save_dir, conf_threshold=0.3):
    model = YOLO(model_path)
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    os.makedirs(save_dir, exist_ok=True)

    detection_results = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]
        frame_data = []

        for box in results.boxes:
            cls = int(box.cls)
            conf = float(box.conf)
            if cls == 0 and conf > conf_threshold:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                frame_data.append({
                    'bbox': [x1, y1, x2, y2],
                    'confidence': conf
                })

        detection_results.append({
            'frame': frame_idx,
            'detections': frame_data
        })

        frame_idx += 1

    cap.release()
    return detection_results

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
model_path = 'best.pt'
broadcast_video = 'broadcast.mp4'
tacticam_video = 'tacticam.mp4'

broadcast_detections = run_detection(broadcast_video, model_path, 'broadcast_detections')

tacticam_detections = run_detection(tacticam_video, model_path, 'tacticam_detections')


0: 384x640 3 players, 68.5ms
Speed: 16.0ms preprocess, 68.5ms inference, 334.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 59.7ms
Speed: 4.3ms preprocess, 59.7ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 61.0ms
Speed: 3.1ms preprocess, 61.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 46.5ms
Speed: 3.2ms preprocess, 46.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 46.6ms
Speed: 2.6ms preprocess, 46.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 players, 1 referee, 46.5ms
Speed: 2.9ms preprocess, 46.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 goalkeeper, 12 players, 1 referee, 36.5ms
Speed: 3.5ms preprocess, 36.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 1 goalkeeper, 14 players, 1 referee, 34.1ms
Speed:

In [5]:
import json

with open('broadcast_detections.json', 'w') as f:
    json.dump(broadcast_detections, f)

with open('tacticam_detections.json', 'w') as f:
    json.dump(tacticam_detections, f)

print("Detections saved to JSON files.")

✅ Detections saved to JSON files.


In [6]:
import numpy as np

class CentroidTracker:
    def __init__(self, max_distance=50, max_frames_lost=30):
        self.next_track_id = 0
        self.tracks = {}
        self.max_distance = max_distance
        self.max_frames_lost = max_frames_lost

    def update(self, detections):
        objects_centroids = []
        for det in detections:
            x1, y1, x2, y2 = det['bbox']
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            objects_centroids.append(((cx, cy), det['bbox']))

        updated_tracks = {}

        for track_id, data in self.tracks.items():
            min_dist = float('inf')
            best_match = None

            for i, (centroid, bbox) in enumerate(objects_centroids):
                dist = np.linalg.norm(np.array(data['centroid']) - np.array(centroid))
                if dist < min_dist and dist < self.max_distance:
                    min_dist = dist
                    best_match = i

            if best_match is not None:
                centroid, bbox = objects_centroids.pop(best_match)
                updated_tracks[track_id] = {'centroid': centroid, 'bbox': bbox, 'frames_since_seen': 0}

        for centroid, bbox in objects_centroids:
            updated_tracks[self.next_track_id] = {'centroid': centroid, 'bbox': bbox, 'frames_since_seen': 0}
            self.next_track_id += 1

        for track_id, data in self.tracks.items():
            if track_id not in updated_tracks:
                data['frames_since_seen'] += 1
                if data['frames_since_seen'] <= self.max_frames_lost:
                    updated_tracks[track_id] = data

        self.tracks = updated_tracks
        return self.tracks

In [7]:
def track_players(detections):
    tracker = CentroidTracker()
    tracking_results = []

    for frame_data in detections:
        frame_id = frame_data['frame']
        detected_players = frame_data['detections']
        tracked_objects = tracker.update(detected_players)

        for track_id, obj in tracked_objects.items():
            tracking_results.append({
                'frame': frame_id,
                'track_id': track_id,
                'bbox': obj['bbox']
            })

    return tracking_results

In [8]:
broadcast_tracks = track_players(broadcast_detections)

tacticam_tracks = track_players(tacticam_detections)

In [9]:
with open('broadcast_tracks.json', 'w') as f:
    json.dump(broadcast_tracks, f)

with open('tacticam_tracks.json', 'w') as f:
    json.dump(tacticam_tracks, f)

print("Tracking results saved.")

Tracking results saved.


In [10]:
from collections import defaultdict

def extract_motion_features(tracks, max_length=30):
    trajectories = defaultdict(list)
    for track in tracks:
        frame = track['frame']
        track_id = track['track_id']
        x1, y1, x2, y2 = track['bbox']
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        trajectories[track_id].append((frame, cx, cy))

    motion_features = {}

    for tid, points in trajectories.items():
        points.sort()
        deltas = []
        for i in range(1, len(points)):
            dx = points[i][1] - points[i-1][1]
            dy = points[i][2] - points[i-1][2]
            deltas.extend([dx, dy])
        if len(deltas) >= 2:
            vec = deltas[:2*max_length]
            if len(vec) < 2*max_length:
                vec += [0] * (2*max_length - len(vec))
            motion_features[tid] = vec

    return motion_features

In [11]:
broadcast_motion = extract_motion_features(broadcast_tracks)
tacticam_motion = extract_motion_features(tacticam_tracks)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.optimize import linear_sum_assignment
import numpy as np

def match_tracks_by_motion(broadcast_feats, tacticam_feats):
    b_ids = list(broadcast_feats.keys())
    t_ids = list(tacticam_feats.keys())

    b_vectors = [broadcast_feats[tid] for tid in b_ids]
    t_vectors = [tacticam_feats[tid] for tid in t_ids]

    similarity = cosine_similarity(b_vectors, t_vectors)

    cost_matrix = 1 - similarity

    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    matched_ids = {}
    for r, c in zip(row_ind, col_ind):
        global_id = f"player_{r+1}"
        matched_ids[f"broadcast_{b_ids[r]}"] = global_id
        matched_ids[f"tacticam_{t_ids[c]}"] = global_id

    return matched_ids

In [13]:
global_id_map = match_tracks_by_motion(broadcast_motion, tacticam_motion)

import json
with open("global_id_mapping.json", "w") as f:
    json.dump(global_id_map, f, indent=2)

print("Player tracks matched across views.")

Player tracks matched across views.


In [14]:
import cv2

def annotate_video(input_video_path, tracks, global_id_map, view_label, output_path):
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    frame_tracks = {}
    for t in tracks:
        f = t['frame']
        if f not in frame_tracks:
            frame_tracks[f] = []
        frame_tracks[f].append(t)

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        objects = frame_tracks.get(frame_idx, [])
        for obj in objects:
            tid = obj['track_id']
            key = f"{view_label}_{tid}"
            global_id = global_id_map.get(key, None)
            if global_id:
                x1, y1, x2, y2 = obj['bbox']
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                cv2.putText(frame, global_id, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    print(f"Saved annotated video: {output_path}")

In [15]:
annotate_video("broadcast.mp4", broadcast_tracks, global_id_map, "broadcast", "output_broadcast.mp4")
annotate_video("tacticam.mp4", tacticam_tracks, global_id_map, "tacticam", "output_tacticam.mp4")

Saved annotated video: output_broadcast.mp4
Saved annotated video: output_tacticam.mp4
